# Markup languages: HTML and XML 

A markup language is a language that appends characters to either side of values in order to tell what that value means. This book was actually written in a simple markup language called 'markdown' that is used in lots of places including Jupyter notebooks. For markdown, doing something like **make this bold** means that the characters were encased in two asterisks like so: \*\*make this bold\*\*. For markup languages that end in ML, like XML and HTML, typically the content is encased in tags like "```<this>```" and "```</this>```". The one with the ```/``` is the ending tag. 

HTML is the markup language used all over the web. Sometimes the content is stored on a server in HTML but often times it is stored in a database and rendered as HTML for the user. For example, it should be obvious that Google does not store a unique HTML page for each person for each query. Instead, it has a vast database of links and has algorithms to transform those into a page on demand. The page that is sent to the browser contains primarily HTML but the data came from elsewhere. Such HTML data it might look something like the following: 

~~~ html
    <html>
        <head>
            <title> 
                This is the title! 
            </title>
        </head>
        <body>
            This is a webpage!
        </body>
    </html>
~~~

If you were to copy that text, paste it into a plain text file with the extension ```.html``` and open it in a browser you will see a blank page with the title bar saying "This is the title!" and a single line saying "This is a webpage!" in the plain, default format. Most functional webpages are vastly more complex but many of the ideas are the same. 

XML is like HTML but the tags will be different. XML stands for eXtensible Markup Language. This is because XML is just a generic standard but it can be extended in a host of ways depending on the needs of the user. For example, in network analysis, we often use a format called GraphML, which is just XML with a special schema for networks that have special characters for things like a 'node' and an 'edge'. As another example, when you download data from Wikipedia it will be returned in XML format with tags for things like the ID of the revision, the date of the last revision, and the content of the Wikipedia page.

Getting data from a webpage or XML into a DataFrame is considerably more tricky than using JSON. This is because you will have to build the row from scratch. Fortunately, there are a number of ways to ease this process. The one that we will use is with a Python library called ```beautifulsoup```. This library will do a lot of the parsing of a webpage for you. The trick is being able to query beautifulsoup effectively so that you get the data you want. This can then be a powerful way to parse pages and data.  

In Chapter XX we will cover some of the complexity of getting data from the web, including ways of getting data from authenticated sources. For now, however, we will gloss over much of the details and work on webpages and XML documents that have been saved for you. Both the webpage and the XML documents are available online free of charge. 

## Basics of beautifulsoup
BeautifulSoup takes in a blob of markedup text and parses it for use. When you use the library it is convention to call the parsed text a ```soup```. We use a soup to help us find text that could be anywhere on the page. Since XML and HTML documents are hierarchical, if we did not have this ability we would have to navigate through the hierarchy. In the above example of HTML, getting the text from the title hierarchically would be ```soup.html.head.title.text``` however, the soup knows that title is a tag so you can just ask for ```soup.title.text``` and it will return ```"This is the title!"```. This ability to just look for tags is especially useful for things like looking for links (which all start with the ```<a>``` tag, as in ```<a href="www.duckduckgo.com">Search with DuckDuckGo</a>```.

To look at beautiful soup, let's first have a look at an HTML page and then an XML data file. We are going to use Wikipedia in both cases and use the exact same page so we can see the difference between the data as seen on the web and the data as seen in HTML. Both files refer to the "Canada" page on Wikipedia. Simply download these two files and place them in the Data folder for your code to run properly.

## Wikipedia as a data source 

In my courses and my research I lean a lot on data from Wikipedia. It is truly a marvel of the Internet age. The accuracy of pages on Wikipedia is often high caliber and the data that is available from the site is often staggering in its depth. In past work I have made use of Wikipedia pages, pages for authors, statistics for page views and edits, pages in different languages, and more. In research I like to suggest that Wikipedia is a great place to start but a terrible place to end. This means an emphasis on critically engaging the content as well as checking out the sources. 

One of the nice things about Wikipedia is that as a freely accessible encyclopedia, there's always content that can be used in teaching and research. In this chapter we will use a snapshot of a Wikipedia page that has been stored in the Data file. We will compare that snapshot as formatted HTML as well as unformatted XML with wikitext. In later chapters, we will revisit Wikipedia for plotting temporal data as well as practising one of data science's most slippery techniques: the regular expression.  For now, we are going to use beautifulsoup and some pandas methods to simply explore the structure of a Wikipedia page and get some basic statistics about its features. This will simultaneously show how to work with some html data and how to build a DataFrame. 

## Wikipedia as HTML 
On the web, Wikipedia is formatted as HTML. It has links that go both within Wikipedia as well as links that go to other sites. The page will have a consistent format regardless of the Wikipedia entry. You can see the underlying text that we are working with by opening ```Canada_Wiki.html``` in a text editor, or see it formatted by opening it in a web browser. The page should look similar to ```https://en.wikipedia.org/wiki/Canada``` although the live page will undoubtedly have at least a few tweaks to the content between when the book was published and when you look at the page.

In [9]:
import bs4,os 

wikiHTML = open("..{0}Data{0}Canada_Wiki.html".format(os.sep),'r').read()
print(len(wikiHTML))

940053


At this point ```wikiHTML``` is just raw text. Printing the length shows it is a long series of characters, so it is probably the page as expected. We can preview the text by printing a range of characters such as ```print(wikiHTML[:200])``` for the first 200 characters. This gets as far as showing the title of the page is Canada. So far, so good. 

In [11]:
print(wikiHTML[:200])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Canada - Wikipedia</title>
<script>document.documentElement.className=document.documentElement.clas


## Using BeautifulSoup 

We Beautifulsoup to parse the HTML for us. This means that it will create a 'soup' which is an object that allows us to query for things in the HTML hierarchy without necessarily going step by step. Recall above in the simple HTML example that the text for the title was nested ```<html><head><title>```? Well with a soup you can just ask for soup.title.text instead of soup.html.head.title.text. This is pretty handy for sourcing links and sections from an HTML page. See below how we first parse the page, print the title text, and look for links.  

In [12]:
# Step 1. Make the soup 
soup = bs4.BeautifulSoup(wikiHTML, 'html.parser')

# Query the soup
print(soup.title.text)
links = soup.find_all("a")
print(len(links))

Canada - Wikipedia
3920


This procedure came up with 3920 unique links in the HTML page for Canada. Yikes, even for a single page on Wikipedia, we are already getting into scales that would be hard for people to work with on their own. Imagine trying to count all those links by hand, for even a handful of versions of the page. 

Although it says that there are 3920 links on this page, I think we ought to practise some data skepticism here. Are URLs the only links available? One way to check is to parse through them and check that the result of the link contains both ```href``` ("hypertext reference") in the HTML tag and ```http``` in the URL string. If they don't contain ```href``` then they are probably either accidents or links to different sections within the page.

In [16]:
urls = []
internal_links = []

for souplink in soup.find_all('a'):
    link = souplink.get('href')
    if link: # That means the link is a hypertext reference and not a section heading
        if 'http' in link:
            urls.append(link)
        else:
            internal_links.append(link) 
    else:
        print(souplink)

print(len(urls),len(internal_links))

<a id="top"></a>
<a class="mw-selflink selflink">Canada</a>
<a class="mw-selflink selflink">Canada</a>
<a class="mw-selflink selflink">Canada</a>
919 2997


So we have discovered here that the page contains 919 links that have ```http``` and are thus likely to be external links elsewhere. There are also a whopping 2997 links to somewhere else either on the page or within the same domain. We can have a preliminary look at the internal links to see what sort of data we have. 

In [19]:
for i in internal_links[:10]: print(i)

/wiki/Wikipedia:Featured_articles
/wiki/Wikipedia:Protection_policy#semi
/wiki/File:En-Canada.ogg
#mw-head
#p-search
/wiki/Canada_(disambiguation)
/wiki/Geographic_coordinate_system
//tools.wmflabs.org/geohack/geohack.php?pagename=Canada&params=60_N_95_W_
/wiki/File:Flag_of_Canada_(Pantone).svg
/wiki/Flag_of_Canada


This shows that the links appear to be pointing to a variety of locations and file types, including links on the Canada page itself, audio files, links to other relevant pages, etc. Presently, I do not have a specific research question about these links, but we can come up with one by thinking about comparing these links across pages. For example, some ideas might be to ask:
- Between countries, which have more links to external government websites? 
- For the same country in different languages, are there similar levels of content? In this case, looking at Canada in English and French (the two official languages) would be revealing. 
- Over time, has the number of links increased? Did it level off after a certain point? 

These are left as puzzles for you, and more will be available in the exercises. But to get you started, I will show here a little snippet for how to get these links into a table and then use map/lambda to detect which links are to internal wikis.

In [36]:
wikiLinks = pd.DataFrame(internal_links,columns=["internal_links"])

def get_wiki(text):
    if text[:5] == "/wiki": return True
    else: return False
    
wikiLinks["wiki"] = wikiLinks["internal_links"].map(lambda x: get_wiki(x))
wikiLinks.head(10)

print("There are {} internal links on this page, {} of which are unique, and {} of which are to other wiki pages".format( 
        len(wikiLinks["internal_links"]), 
        len(wikiLinks["internal_links"].unique() ),
        len(wikiLinks[wikiLinks["wiki"]]) #Notice here I sliced to only "wiki" == True.
        ))

There are 2997 internal links on this page, 2355 of which are unique, and 2035 of which are to other wiki pages


## Using Pandas read_table method
Before moving on, I wanted to briefly illustrate how parsing HTML can be a joy and a curse. The pandas ```read_table``` method uses beautifulsoup to find tables in an HTML document and then parse them. However, it does not work great. Looking at some of the ways it did not work out-of-the-box can help you appreciate some of the challenges that lie ahead in properly parsing data. Don't get me wrong, it is pretty nifty and with some tweaking you might get it working for you, but these are the sorts of things that highlight how this work can be messy and requires significant finesse.

Applying ```read_table``` to the Canada page gets us a table that corresponds to the infobox in the upper right corner. Unfortunately, the parser by default seems to struggle with many of the other tables on the page. 

In [37]:
soup = bs4.BeautifulSoup(wikiHTML,'lxml') #res.content
tables = soup.find_all('table')[0] 
parsed_tables = pd.read_html(str(tables)) # This will return a list of DataFrames, one for each table detected.
print(len(parsed_tables)) # This will show us there is only one table detected. 
display(parsed_tables[0].head())

1


,Canada,Canada.1
0,Flag Coat of arms,Flag Coat of arms
1,"Motto: A Mari Usque Ad Mare (Latin)""From Sea t...","Motto: A Mari Usque Ad Mare (Latin)""From Sea t..."
2,"Anthem: ""O Canada""[a]","Anthem: ""O Canada""[a]"
3,NaN,NaN
4,Capital,Ottawa45°24′N 75°40′W﻿ / ﻿45.400°N 75.667°W


So showing this we can see that the data is still really messy. The column headers have the names "Canada" and "Canada.1", there is repetition in the text and some empty text. In the 5th row (index[4]) we see Capital in one column and then "Ottawa45°24′N 75°40′W﻿ / ﻿45.400°N 75.667°W" in the next, meaning that it did clearly did not record a return character between Ottawa and a geocode. This is a great example of why much of data science is actually wrangling. The data you know is there, but it is in a messy format and needs to be cleaned. 

In the next chapter we will look at some approaches to cleaning and detecting different features in the data. We will use regular expressions to see if we can detect URLs directly as well as other features like the latitude and longitude coordinates seen above. In the meantime, we will look at the raw XML for this page as well as a host of other data types. 